In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

text = """
Character-level language modeling is a text generation task where the model predicts the next character in a sequence of characters.
Recurrent Neural Networks (RNNs) are suitable for this task, as they can capture sequential dependencies.
"""

text = text.lower()
tokenizer = Tokenizer(char_level=True)
tokenizer.fit_on_texts(text)
total_chars = len(tokenizer.word_index) + 1

input_sequences = []
target_chars = []
seq_length = 100

for i in range(0, len(text) - seq_length, 1):
    input_seq = text[i:i + seq_length]
    target_seq = text[i + seq_length]
    input_sequences.append(input_seq)
    target_chars.append(target_seq)

char_to_index = tokenizer.word_index
X = np.array([[char_to_index[char] for char in seq] for seq in input_sequences])

y = to_categorical([char_to_index[char] for char in target_chars], num_classes=total_chars)

model = Sequential()
model.add(Embedding(total_chars, 128, input_length=seq_length))
model.add(LSTM(128))
model.add(Dense(total_chars, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

model.fit(X, y, epochs=50, batch_size=64)

seed_text = "character-level language modeling is"
generated_text = seed_text

for _ in range(200):
    x = np.array([[char_to_index[char] for char in seed_text]])
    x = pad_sequences(x, maxlen=seq_length, truncating='pre', value=char_to_index[' '])
    predicted_char_index = np.argmax(model.predict(x))
    predicted_char = [char for char, index in char_to_index.items() if index == predicted_char_index][0]
    generated_text += predicted_char
    seed_text = seed_text[1:] + predicted_char

print(generated_text)

Epoch 1/50
3/3 [==============================] - 9s 455ms/step - loss: 3.4610
Epoch 2/50
3/3 [==============================] - 2s 474ms/step - loss: 3.4283
Epoch 3/50
3/3 [==============================] - 2s 511ms/step - loss: 3.3498
Epoch 4/50
3/3 [==============================] - 1s 290ms/step - loss: 3.0425
Epoch 5/50
3/3 [==============================] - 1s 306ms/step - loss: 2.9698
Epoch 6/50
3/3 [==============================] - 1s 238ms/step - loss: 2.9185
Epoch 7/50
3/3 [==============================] - 1s 201ms/step - loss: 2.8961
Epoch 8/50
3/3 [==============================] - 0s 149ms/step - loss: 2.8913
Epoch 9/50
3/3 [==============================] - 0s 140ms/step - loss: 2.8834
Epoch 10/50
3/3 [==============================] - 0s 133ms/step - loss: 2.8649
Epoch 11/50
3/3 [==============================] - 0s 130ms/step - loss: 2.8527
Epoch 12/50
3/3 [==============================] - 0s 135ms/step - loss: 2.8526
Epoch 13/50
3/3 [==============================] 

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split


sentences = [
    "Apple Inc. is headquartered in Cupertino, California.",
    "John Smith works at Google in New York.",
    "Paris is a beautiful city in France."
]

labels = [
    "ORG ORG O O O LOC LOC O O",
    "PER PER O O ORG O LOC LOC O",
    "LOC O O O LOC LOC LOC O"
]

max_seq_length = 15
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)
vocab_size = len(tokenizer.word_index) + 1

X = tokenizer.texts_to_sequences(sentences)
X = pad_sequences(X, maxlen=max_seq_length, padding='post')

label_tokenizer = Tokenizer(filters=' ')
label_tokenizer.fit_on_texts(labels)
num_classes = len(label_tokenizer.word_index) + 1

y = label_tokenizer.texts_to_sequences(labels)
y = pad_sequences(y, maxlen=max_seq_length, padding='post')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=128, input_length=max_seq_length))
model.add(LSTM(128, return_sequences=True))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

y_train = [tf.keras.utils.to_categorical(label, num_classes=num_classes) for label in y_train]

model.fit(X_train, np.array(y_train), epochs=10, batch_size=32)

y_test = [tf.keras.utils.to_categorical(label, num_classes=num_classes) for label in y_test]
loss, accuracy = model.evaluate(X_test, np.array(y_test))

print(f"Test loss: {loss:.4f}")
print(f"Test accuracy: {accuracy:.4f}")



Epoch 1/10
1/1 [==============================] - 5s 5s/step - loss: 1.6193 - accuracy: 0.1000
Epoch 2/10
1/1 [==============================] - 0s 40ms/step - loss: 1.5937 - accuracy: 0.6000
Epoch 3/10
1/1 [==============================] - 0s 28ms/step - loss: 1.5680 - accuracy: 0.5667
Epoch 4/10
1/1 [==============================] - 0s 36ms/step - loss: 1.5412 - accuracy: 0.5667
Epoch 5/10
1/1 [==============================] - 0s 29ms/step - loss: 1.5123 - accuracy: 0.5667
Epoch 6/10
1/1 [==============================] - 0s 31ms/step - loss: 1.4804 - accuracy: 0.5667
Epoch 7/10
1/1 [==============================] - 0s 36ms/step - loss: 1.4443 - accuracy: 0.5333
Epoch 8/10
1/1 [==============================] - 0s 33ms/step - loss: 1.4029 - accuracy: 0.5333
Epoch 9/10
1/1 [==============================] - 0s 35ms/step - loss: 1.3550 - accuracy: 0.5000
Epoch 10/10
1/1 [==============================] - 1s 859ms/step - loss: 1.2747 - accuracy: 0.4667
Test loss: 1.2747
Test accurac